# Linear Adjustments

## Critical quantities

These 5 values need to be determined every time

* $n$: The number of observations (they are uncertain i.e. have covariance $\Sigma$ -- which is the point).
* $n_0$: The number of values (observations or parameters) that need to be fixed to determine 'the model'.
* $r$: Redundancy: $r=n-n_0$.
* $u$: Number of unknown parameters.
* $c$: Number of condition equations: $c=r+u$.

The first 3 are determined analytically, then $u$ and $c$ flow from them.

## Method of Indirect Observations
In this method we solve for the parameters, so $u$ is that number. For instance for fitting a line, parameters are slope and intercept $m,b$, so $u=2$.

Once the $c=r+u$ condition equations are written, with all uncertain observations $\hat{l}_i$ expanded to solved observations plus residuals $l_i + v_i$, the linear system is formed/rearranged as $$v + B\Delta=f$$

Then the solution method is this series of matrix calculations:

* Weight matrix $W=\Sigma^{-1}$
* Normal equations: $N=B'WB$ and $t=B'Wf$
* Parameter solution: solve $N\Delta=t \Rightarrow \Delta = N^{-1}t$
* Residuals: $v=f-B\Delta$
* Covariance of parameters: $\Sigma_{\Delta\Delta} = N^{-1}$
* Covariance of residuals: $\Sigma_{vv} = \Sigma - BQ_{\Delta\Delta}B'$
* Adjusted observations: $\hat{l} = -fN' + v$
* Covariance of $\hat{l}$: $\Sigma_{\hat{l}\hat{l}}=\Sigma-\Sigma{vv}$




In [1]:
import numpy as np

In [50]:
def print_indirect_observations(B, f, l=None, Q=None):
    c,u = B.shape
    r = c-u
    n0 = u      # always for indirect observations
    n  = n0 + r # always
    print('Problem shape:')
    print('  n:', n)
    print(' n0:', n0)
    print('  r:', r)
    print('  u:', u)
    print('  c:', c)
    
    print('Problem:')
    print('B:\n', B)
    print('f:', f.transpose())
    
    print('Normal Equations:')
    if Q is None: Q = np.eye(n)
    W = np.linalg.inv(cov)
    print('Weight:\n',W)
    N = B.transpose() @ W @ B
    t = B.transpose() @ W @ f
    print('N:\n', N)
    print('t:', t.transpose())
    
    print('Solution')
    d = np.linalg.solve(N,t)
    print('Parameters:', d.transpose())
    Qdd = np.linalg.inv(N)
    print('Parameter covariance:\n',Qdd) 
    
    v = f - B@d
    print('Residuals:', v.transpose())
    Qvv = Q - B @ Qdd @ B.transpose()
    print('Residual covariance:\n', Qvv)
    
    if l is not None:
        lhat = l + v
        print('Original observations:', l.transpose())
        print('Adjusted observations:', lhat.transpose())
        Qll = Q - Qvv
        print('Covariance:\n',Qll)
    

Line fit example: points (1,1.1), (2,1.2), (3,1.5), (4,1.9) with $s_i=0.1$. 

* $n=4$: the four $y_i$
* $n_0=2$: line parameters $m,b$
* $r=2$
* $u=2$
* $c=4$, the line equations $\hat{y_i}=(y_i+v_i) = mx_i + b$

Condition equations reshape to $$v_i + -mx_i - b = -y_i$$

In [56]:
B = np.array([ [-1,-1], [-2,-1], [-3,-1], [-4,-1] ])
B

array([[-1, -1],
       [-2, -1],
       [-3, -1],
       [-4, -1]])

In [57]:
f = np.array([-1.1, -1.2, -1.5, -1.9]).reshape( (4,1) )
f

array([[-1.1],
       [-1.2],
       [-1.5],
       [-1.9]])

In [58]:
cov = np.eye(4) * 0.1*0.1

In [59]:
print_indirect_observations(B, f, -f, cov)

Problem shape:
  n: 4
 n0: 2
  r: 2
  u: 2
  c: 4
Problem:
B:
 [[-1 -1]
 [-2 -1]
 [-3 -1]
 [-4 -1]]
f: [[-1.1 -1.2 -1.5 -1.9]]
Normal Equations:
Weight:
 [[100.   0.   0.   0.]
 [  0. 100.   0.   0.]
 [  0.   0. 100.   0.]
 [  0.   0.   0. 100.]]
N:
 [[3000. 1000.]
 [1000.  400.]]
t: [[1560.  570.]]
Solution
Parameters: [[0.27 0.75]]
Parameter covariance:
 [[ 0.002 -0.005]
 [-0.005  0.015]]
Residuals: [[-0.08  0.09  0.06 -0.07]]
Residual covariance:
 [[ 0.003 -0.004 -0.001  0.002]
 [-0.004  0.007 -0.002 -0.001]
 [-0.001 -0.002  0.007 -0.004]
 [ 0.002 -0.001 -0.004  0.003]]
Original observations: [[1.1 1.2 1.5 1.9]]
Adjusted observations: [[1.02 1.29 1.56 1.83]]
Covariance:
 [[ 0.007  0.004  0.001 -0.002]
 [ 0.004  0.003  0.002  0.001]
 [ 0.001  0.002  0.003  0.004]
 [-0.002  0.001  0.004  0.007]]
